In [149]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

df = pd.read_csv('../data/text_data.csv')

stop_words = {"и", "в", "во", "не", "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", "она", "так", "его",
                  "но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по", "только", "ее", "мне", "было", "вот", "от",
                  "меня", "еще", "нет", "о", "из", "ему", "теперь", "когда", "даже", "ну", "вдруг", "ли", "если", "уже",
                  "или", "ни", "быть", "был", "него", "до", "вас", "нибудь", "опять", "уж", "вам", "ведь", "там",
                  "потом", "себя", "ничего", "ей", "может", "они", "тут", "где", "есть", "надо", "ней", "для", "мы",
                  "тебя", "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего", "раз", "тоже", "себе", "под",
                  "будет", "ж", "тогда", "кто", "этот", "того", "потому", "этого", "какой", "совсем", "ним", "здесь",
                  "этом", "почти", "мой", "тем", "чтобы", "нее", "сейчас", "были", "куда", "зачем", "всех", "никогда",
                  "можно", "при", "наконец", "два", "об", "другой", "хоть", "после", "над", "больше", "тот", "через",
                  "эти", "нас", "про", "всего", "них", "какая", "много", "разве", "три", "эту", "моя", "впрочем",
                  "хорошо", "свою", "этой", "перед", "иногда", "лучше", "чуть", "том", "нельзя", "такой", "им", "более",
                  "всегда", "конечно", "всю", "между"}

df

,Тематика,Текст обращения,Количество комментариев,Количество репостов
0,Здравоохранение,NaN,61,50
1,Здравоохранение,Прошу рассмотреть вопрос о ремонте крыши в мно...,92,43
2,Культура,"Транспорт ходит с большими задержками, особенн...",14,43
3,Транспорт,Погибли бездомные животные из-за отравления. К...,94,28
4,Безопасность,Где обещанная модернизация больницы? Оборудова...,77,5
...,...,...,...,...
495,Образование,"Нет воды в доме номер 12 уже вторые сутки, про...",73,44
496,NaN,Прошу рассмотреть вопрос о ремонте крыши в мно...,69,46
497,Культура,Примите меры! В школе не хватает учебников для...,25,45
498,NaN,"Глава региона, прошу рассмотреть вопрос о стро...",38,0


In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Тематика                 479 non-null    object
 1   Текст обращения          460 non-null    object
 2   Количество комментариев  500 non-null    int64 
 3   Количество репостов      500 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 15.8+ KB


In [151]:
df['Тематика'] = df['Тематика'].fillna('Не указано')
df = df.dropna(subset=['Текст обращения']).copy()
num1 = pd.DataFrame(df['Текст обращения'].unique())
num1

,0
0,Прошу рассмотреть вопрос о ремонте крыши в мно...
1,"Транспорт ходит с большими задержками, особенн..."
2,Погибли бездомные животные из-за отравления. К...
3,Где обещанная модернизация больницы? Оборудова...
4,"Уважаемая администрация, прошу обратить вниман..."
5,Крик души! В поликлинике номер 5 огромные очер...
6,Примите меры! В школе не хватает учебников для...
7,"Глава региона, прошу рассмотреть вопрос о стро..."
8,"Нет воды в доме номер 12 уже вторые сутки, про..."
9,От лица жителей обращаюсь по поводу отсутствия...


In [152]:
print(f'Количество дубликатов: {df.duplicated().sum()}')
print('Пропущенные значения:')
df.isna().sum().sort_values(ascending=False)

Количество дубликатов: 0
Пропущенные значения:


Тематика                   0
Текст обращения            0
Количество комментариев    0
Количество репостов        0
dtype: int64

In [153]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    word = [word for word in words if word not in stop_words]

    return " ".join(word)
df['Текст обращения'] = df['Текст обращения'].apply(clean_text)

In [154]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['Текст обращения'])

num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=143, n_init=10)
df['Кластер'] = kmeans.fit_predict(X)
df['Кластер'].value_counts()

Кластер
2    59
5    54
3    50
4    48
1    45
7    45
6    44
0    44
8    42
9    29
Name: count, dtype: int64

In [155]:
for cluster_num in range(num_clusters):
    print(f'Кластер: {cluster_num}')
    print(df[df['Кластер'] == cluster_num]['Текст обращения'].head().tolist())
    print('-' * 50)

Кластер: 0
['обещанная модернизация больницы оборудование устарело врачи справляются', 'обещанная модернизация больницы оборудование устарело врачи справляются', 'обещанная модернизация больницы оборудование устарело врачи справляются', 'обещанная модернизация больницы оборудование устарело врачи справляются', 'обещанная модернизация больницы оборудование устарело врачи справляются']
--------------------------------------------------
Кластер: 1
['воды доме номер вторые сутки прошу срочно разобраться', 'воды доме номер вторые сутки прошу срочно разобраться', 'воды доме номер вторые сутки прошу срочно разобраться', 'воды доме номер вторые сутки прошу срочно разобраться', 'воды доме номер вторые сутки прошу срочно разобраться']
--------------------------------------------------
Кластер: 2
['примите меры школе хватает учебников учеников х классов', 'примите меры школе хватает учебников учеников х классов', 'примите меры школе хватает учебников учеников х классов', 'примите меры школе хвата

In [156]:
cluster_names = {
    0: 'Проблемы с больницей',
    1: 'Проблемы с трубопроводом',
    2: 'Нехватка учебников в школе',
    3: 'Строительство детской площадки',
    4: 'Проблемы с освещением',
    5: 'Гибель бездомных животных',
    6: 'Огромные очереди в поликлинике',
    7: 'Задержка транспорта',
    8: 'Состояние дорог',
    9: 'Ремонт крыши',
}
df['Тематика'] = df['Кластер'].map(cluster_names)
df.drop(columns=['Кластер'], inplace=True)

df

,Тематика,Текст обращения,Количество комментариев,Количество репостов
1,Ремонт крыши,прошу рассмотреть вопрос ремонте крыши многокв...,92,43
2,Задержка транспорта,транспорт ходит большими задержками особенно в...,14,43
3,Гибель бездомных животных,погибли бездомные животные изза отравления отв...,94,28
4,Проблемы с больницей,обещанная модернизация больницы оборудование у...,77,5
6,Состояние дорог,уважаемая администрация прошу обратить внимани...,11,3
...,...,...,...,...
495,Проблемы с трубопроводом,воды доме номер вторые сутки прошу срочно разо...,73,44
496,Ремонт крыши,прошу рассмотреть вопрос ремонте крыши многокв...,69,46
497,Нехватка учебников в школе,примите меры школе хватает учебников учеников ...,25,45
498,Строительство детской площадки,глава региона прошу рассмотреть вопрос строите...,38,0


In [157]:
print(f'Количество дубликатов: {df.duplicated().sum()}')
print('Пропущенные значения:')
df.isna().sum().sort_values(ascending=False)

Количество дубликатов: 1
Пропущенные значения:


Тематика                   0
Текст обращения            0
Количество комментариев    0
Количество репостов        0
dtype: int64

In [158]:
df = df.drop_duplicates()

In [159]:
print('Статистика по кластерам:')
df['Тематика'].value_counts()

Статистика по кластерам:


Тематика
Нехватка учебников в школе        58
Гибель бездомных животных         54
Строительство детской площадки    50
Проблемы с освещением             48
Проблемы с трубопроводом          45
Задержка транспорта               45
Огромные очереди в поликлинике    44
Проблемы с больницей              44
Состояние дорог                   42
Ремонт крыши                      29
Name: count, dtype: int64

In [160]:
df.to_csv('../data/cleaned_text_data.csv', index=False)